In [173]:
import numpy as np
import pandas as pd
import pymc3 as pm
import arviz as az
import theano.tensor as tt
import scipy as sp
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

In [174]:
# getting the data

data = pd.read_csv('cleaned_data.csv' , sep = ' ' , header = None)
data

,0,1,2,3,4
0,0,0,0,0,0
1,0,0,0,0,0
2,0,0,0,0,0
3,0,0,0,0,1
4,0,0,0,0,1
...,...,...,...,...,...
995,1,1,1,1,1
996,1,1,1,1,1
997,1,1,1,1,1
998,1,1,1,1,1


Rasch Model

$$\Pr(X_{ij}=1) = \frac{e^{\alpha_j - \beta\times\theta_i}}{1 + e^{\alpha_j - \beta\times\theta_i}}$$

$\beta_n$ = ability of student n

$\delta_i$ = difficulty of item i

https://towardsdatascience.com/a-bayesian-approach-to-rasch-models-item-response-theory-cc08805cbb37

In [175]:
# define some constants according to the constraints in the question
N = 1000
Q = 5
sigma_alpha = 100
mean_alpha = 0
mean_beta = 0
sigma_beta = 1
mean_theta = 0
sigma_theta = 1

On doing some simple mathematics, we get the likelihood to be:

$\mathcal{L}(x | \{\alpha_j\}_{j=1}^Q , \{\theta_i\}_{i=1}^N , \beta) = \frac{\exp\left(\beta\sum\limits_{i=1}^N \theta_i r_i - \sum\limits_{j=1}^Q\alpha_j s_j\right)}{\prod\limits_{i=1}^Q\prod\limits_{j=1}^N 1 + \exp\left({\beta\theta_i - \alpha_j}\right)}$

where $r_i$ is the score of the $i^{th}$ student and $s_j$ is the score achieved on the $j^{th}$ question.


We also have our priors (all the means are zero):

$\pi(\alpha_j) = \frac{1}{\sqrt{2\pi\sigma_\alpha^2}} \exp\left({-\frac{\alpha_j^2}{2\sigma_\alpha^2}}\right)$

$\pi(\theta_i) = \frac{1}{\sqrt{2\pi\sigma_\theta^2}} \exp\left({-\frac{\theta_i^2}{2\sigma_\theta^2}}\right)$

$\pi(\beta) = \frac{1}{\sqrt{2\pi\sigma_\beta^2}} \exp\left({-\frac{\beta^2}{2\sigma_\beta^2}}\right) \times \mathbf{1}\left(\beta \geq 0\right)$

Thus, the posterior distribution is given by:

$\pi(\{\alpha_j\}_{j=1}^Q , \{\theta_i\}_{i=1}^N , \beta | x) = \mathcal{L}(x | \{\alpha_j\}_{j=1}^Q , \{\theta_i\}_{i=1}^N , \beta) \times \prod\limits_{j=1}^Q\pi(\alpha_j) \times \prod\limits_{i=1}^N\pi(\theta_i) \times \pi(\beta)$

For more details about the mathematics, please refer to the paper.

Since we wish to sample for $\{\alpha_j\}_{j=1}^Q , \{\theta_i\}_{i=1}^N , \beta$ , we would have to obtain the  conditional distributions of $\alpha_j , \theta_i , \beta$.

$\pi(\alpha_j | \alpha_1 , \ldots , \alpha_{j-1} , \alpha_{j+1} , \ldots, \alpha_Q , \{\theta_i\}_{i=1}^N , \beta, x) = \frac{\exp\left(-\alpha_j s_j - \frac{\alpha_j^2 }{2\sigma_\alpha^2}\right)}{\prod\limits_{i=1}^Q\prod\limits_{j=1}^N 1 + \exp\left({\beta\theta_i - \alpha_j}\right)}$

$\pi(\beta | \{\alpha_j\}_{j=1}^Q , \{\theta_i\}_{i=1}^N , x) = \frac{\exp\left(\beta\sum\limits_{i=1}^N \theta_i r_i  - \frac{\beta^2 }{2\sigma_\beta^2}\right)}{\prod\limits_{i=1}^Q\prod\limits_{j=1}^N 1 + \exp\left({\beta\theta_i - \alpha_j}\right)}$

$\pi(\theta_i | \theta_1 , \ldots , \theta_{i-1} , \theta_{i+1} , \ldots, \theta_N , \{\alpha_j\}_{j=1}^Q , \beta, x) = \frac{\exp\left(\beta\theta_i r_i  - \frac{\theta_i^2 }{2\sigma_\theta^2}\right)}{\prod\limits_{i=1}^Q\prod\limits_{j=1}^N 1 + \exp\left({\beta\theta_i - \alpha_j}\right)}$

In [176]:
score_per_question = np.sum(data, axis = 0)
score_per_student = np.sum(data, axis = 1)

In [177]:
def initialize_chain():
    alpha = np.random.normal(mean_alpha , sigma_alpha , size = Q)
    alpha -= alpha.mean()

    # halfFlat is a uniform distribution between 0 and infinity
    # beta = 1

    beta = np.random.normal(mean_beta , sigma_beta)
    beta *= (beta >= 0)

    theta = np.random.normal(mean_theta , sigma_theta , size = N)
    return alpha, beta, theta

In [178]:
def log_den(alpha , beta , theta):
    log_den = np.float(0)
    for i in range(N):
        for j in range(Q):
            log_den += np.log(1 + np.exp(beta * theta[i] - alpha[j]))
    return log_den

def target_alpha(alpha, beta, theta , j , log_den):
    m = (-alpha[j] * score_per_question[j] - 0.5 * (alpha[j]/sigma_alpha)**2 - log_den)
    return m

def target_beta(alpha, beta, theta , log_den):
    m = (np.sum(np.dot(theta , score_per_student)) - 0.5 * (beta/sigma_beta)**2 - log_den)
    return m

def target_theta(alpha, beta, theta , i , log_den):
    m = (beta * theta[i] * score_per_student[i] - 0.5 * (theta[i]/sigma_theta)**2 -  log_den)
    return m

In [179]:
def mcmc_sampler(num_chains , num_steps, burn_in):
    eps = 1e-6
    alpha_samples = []
    beta_samples = []
    theta_samples = []

    for chain_num in range(num_chains):
        alpha , beta , theta = initialize_chain()

        alpha_chain = []
        beta_chain = []
        theta_chain = []

        
        # we assume proposal distribution to be symmetric
        for step in range(num_steps):

            proposal_alpha = lambda x: np.random.normal(x , sigma_alpha)
            proposal_beta = lambda x: np.random.normal(x , sigma_beta)
            proposal_theta = lambda x: np.random.normal(x , sigma_theta)

            alpha_new = np.array([proposal_alpha(alpha_j) for alpha_j in alpha])
            alpha_new -= alpha_new.mean()
            beta_new = proposal_beta(beta)
            beta_new = beta_new * (beta_new >= 0)
            theta_new = np.array([proposal_theta(theta_i) for theta_i in theta])

            print(alpha_new)
            print(beta_new)

            target_new = 0
            target_old = 0
            log_den_new = log_den(alpha_new , beta_new , theta_new)
            log_den_old = log_den(alpha , beta , theta)

            # print(log_den_new , log_den_old)
            for j in range(Q):
                target_new += target_alpha(alpha_new, beta_new, theta_new , j , log_den_new)
                target_old += target_alpha(alpha, beta, theta , j , log_den_old)
            
            for i in range(N):
                target_new += target_theta(alpha_new, beta_new, theta_new , i , log_den_new)
                target_old += target_theta(alpha, beta, theta , i , log_den_old)
            
            target_new += target_beta(alpha_new, beta_new, theta_new , log_den_new)
            target_old += target_beta(alpha, beta, theta , log_den_old)
            # print(target_new , target_old)
            acceptance_prob = target_new/target_old
            print('(Chain: {}, Step: {}/{})=> Acceptance Probability = {:.4f}'.format(chain_num+1, step+1, num_steps, acceptance_prob))

            u = np.random.rand()
            if u < min(1 , acceptance_prob):
                if step > burn_in-1:
                    alpha_chain.append(alpha_new)
                    beta_chain.append(beta_new)
                    theta_chain.append(theta_new)
                
                alpha = alpha_new
                beta = beta_new
                theta = theta_new

            else:
                alpha_chain.append(alpha)
                beta_chain.append(beta)
                theta_chain.append(theta)

        alpha_samples.append(alpha_chain)
        beta_samples.append(beta_chain)
        theta_samples.append(theta_chain)
        print('\n')
    return alpha_samples , beta_samples , theta_samples


In [180]:
s_alpha, s_beta, s_theta = mcmc_sampler(1, 50, 5)

[-330.6385405    38.99450865  -70.88015895   66.752635    295.7715558 ]
2.3084106710008787
(Chain: 1, Step: 1/50)=> Acceptance Probability = 1.6787
[-428.05075865  113.12656751 -103.53440786  -17.59194849  436.05054749]
3.0015035663251197
(Chain: 1, Step: 2/50)=> Acceptance Probability = 1.3685
[-429.69952009  116.78314934 -129.10985238   28.05292221  413.97330092]
3.646441127293747
(Chain: 1, Step: 3/50)=> Acceptance Probability = 1.0175
[-435.39547441  118.6729058  -190.56551898   79.56111997  427.72696763]
2.718234390847845
(Chain: 1, Step: 4/50)=> Acceptance Probability = 1.1197
[-408.15370821  211.62653912 -203.39569179  152.36699121  247.55586967]
4.188521917183125
(Chain: 1, Step: 5/50)=> Acceptance Probability = 0.9772
[-468.83265143  154.14807007 -242.29713821   96.87434849  460.10737108]
6.692920769213205
(Chain: 1, Step: 6/50)=> Acceptance Probability = 1.1630
[-470.51869074  232.04919693 -286.20466891   90.72622201  433.94794071]
5.533753057464498
(Chain: 1, Step: 7/50)=> A

In [181]:
means = np.zeros(5)
for i in range(len(s_alpha)):
    means+=np.mean(s_alpha[i], axis = 0)
print(means/len(s_alpha))

[-765.68769971  383.26202745  -87.23551223  451.25895706   18.40222743]


In [182]:
np.array(s_beta).mean()

3.7980396427025216